# 넷플릭스 콘텐츠 기반 추천: TF-IDF 벡터라이저 학습 및 유명 영화 유사도 분석

이 노트북에서는 다음을 수행합니다:

1. `video.csv`에서 적절한 태그 생성 (장르, 설명, 감독, 배우, 국가 등)
2. TF-IDF 벡터라이저 학습 및 저장
3. 유명 영화(예: IMDb 평점, 시청수 등 기준) 선정
4. 해당 영화와 유사한 영화 추천 (코사인 유사도 기반)

---

**실행 전 준비:**
- `video.csv` 파일이 같은 경로에 있어야 합니다.
- 필요한 라이브러리: pandas, numpy, scikit-learn, joblib


In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [11]:
def preprocess_dataframe(df):
    delete_columns = [i for i in df.columns if 'Unnamed' in i]

    df.drop(columns=delete_columns, inplace=True)

    return df

In [12]:
# 데이터 로드
df = pd.read_csv('../video.csv')
df = preprocess_dataframe(df)
df.head()

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,1451,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",24-Sep-21,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
1,1452,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,24-Sep-21,2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
2,1453,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,24-Sep-21,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
3,1454,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",23-Sep-21,2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
4,1455,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,21-Sep-21,1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...


## 1. 태그 생성 함수 정의

장르(listed_in), 설명(description), 감독(director), 주요 배우 2~3명, 국가(country)를 조합해 태그를 만듭니다.

In [13]:
def create_tags(row):
    tags = []
    if pd.notnull(row.get('listed_in')):
        tags.extend(row['listed_in'].split(', '))
    if pd.notnull(row.get('description')):
        tags.extend(row['description'].split())
    if pd.notnull(row.get('director')):
        tags.append(row['director'].replace(' ', '_'))
    # 주요 배우 2~3명
    # if 'cast' in row and pd.notnull(row['cast']):
    #     actors = row['cast'].split(', ')
    #     tags.extend([actor.replace(' ', '_') for actor in actors[:3]])
    # 국가
    if 'country' in row and pd.notnull(row['country']):
        tags.append(row['country'].replace(' ', '_'))
    return ' '.join(tags).lower()

# tags 컬럼 생성
df['tags'] = df.apply(create_tags, axis=1)
df[['title', 'tags']]

,title,tags
0,Sankofa,dramas independent movies international movies...
1,The Great British Baking Show,british tv shows reality tv a talented batch o...
2,The Starling,comedies dramas a woman adjusting to life afte...
3,Je Suis Karl,dramas international movies after most of her ...
4,Jeans,comedies international movies romantic movies ...
...,...,...
5312,Zinzana,dramas international movies thrillers recoveri...
5313,Zodiac,cult movies dramas thrillers a political carto...
5314,Zombieland,comedies horror movies looking to survive in a...
5315,Zoom,children & family movies comedies dragged from...


## 2. TF-IDF 벡터라이저 학습 및 저장

- tags 컬럼을 기반으로 TF-IDF 벡터라이저를 학습합니다.
- 학습된 벡터라이저와 행렬을 저장합니다.

In [14]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['tags'])

# 저장
joblib.dump(tfidf, '../tfidf_vectorizer.joblib')
np.save('../tfidf_matrix.npy', tfidf_matrix.toarray())
print("TF-IDF 벡터라이저와 행렬 저장 완료!")

TF-IDF 벡터라이저와 행렬 저장 완료!


In [15]:
tfidf_matrix.shape

(5317, 5000)

## 3. 유명 영화 선정 및 유사도 기반 추천

- IMDb 평점, 시청수 등으로 유명한 영화를 선정합니다.
- 해당 영화와 코사인 유사도가 높은 영화를 추천합니다.
- (예시: 평점 컬럼이 없으면, title로 유명 영화 직접 지정)

In [18]:
# 예시: 유명 영화 title 직접 지정 (실제 평점/시청수 컬럼이 있으면 그걸로 선정)
# famous_titles = [
#     'Inception', 'The Irishman', 'Roma', 'Marriage Story', 'The Social Dilemma',
#     'Extraction', 'Bird Box', 'The Platform', 'Okja', 'The King'
# ]

famous_titles = df['title'].iloc[:10].values

for title in famous_titles:
    if title in df['title'].values:
        idx = df[df['title'] == title].index[0]
        sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
        sim_indices = sim_scores.argsort()[::-1][1:6]  # 자기 자신 제외 상위 5개
        print(f"\n[유명 영화: {title}]와 유사한 영화 TOP 5:")
        for i in sim_indices:
            print(f"- {df.iloc[i]['title']} (유사도: {sim_scores[i]:.3f})")


[유명 영화: Sankofa]와 유사한 영화 TOP 5:
- When We First Met (유사도: 0.180)
- Secrets of Selfridges (유사도: 0.178)
- The Burial of Kojo (유사도: 0.157)
- Something Huge (유사도: 0.145)
- 16 Blocks (유사도: 0.143)

[유명 영화: The Great British Baking Show]와 유사한 영화 TOP 5:
- Hoff the Record (유사도: 0.360)
- Father Brown (유사도: 0.315)
- London Spy (유사도: 0.308)
- The Twelve (유사도: 0.307)
- Merlin (유사도: 0.296)

[유명 영화: The Starling]와 유사한 영화 TOP 5:
- My Daddy is in Heaven (유사도: 0.210)
- Spivak (유사도: 0.203)
- Chillar Party (유사도: 0.200)
- Recall (유사도: 0.178)
- Escaping Tel Aviv (유사도: 0.173)

[유명 영화: Je Suis Karl]와 유사한 영화 TOP 5:
- Colonia (유사도: 0.228)
- The Cell (유사도: 0.216)
- Who Am I (유사도: 0.215)
- Lechmi (유사도: 0.190)
- One 2 Ka 4 (유사도: 0.190)

[유명 영화: Jeans]와 유사한 영화 TOP 5:
- Sinister 2 (유사도: 0.261)
- Mahi NRI (유사도: 0.226)
- Oh Darling Yeh Hai India (유사도: 0.199)
- Gol Maal (유사도: 0.198)
- The Forest (유사도: 0.196)

[유명 영화: Grown Ups]와 유사한 영화 TOP 5:
- Alelí (유사도: 0.192)
- The Paper Tigers (유사도: 0.188)
- Blue Jay (유사도: 0.174)